In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    data = []
    
    pagination = soup.find("div", class_="styles_pagination__6VmQv")
    last_page = pagination.find_all("a")[-2].text
    last_page = pd.to_numeric(last_page)
    
    for i in range(1, last_page + 1):
        page_url = url + "?page=" + str(i)
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.content, 'html.parser')
        
        reviews = page_soup.find_all("div", class_="styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ")
        
        for review in reviews:
            review_data = {}
            
            review_data["Company"] = soup.find("h1", class_="typography_default__hIMlQ typography_appearance-default__AAY17 title_title__i9V__").span.get_text(strip=True)
            review_data["Customer"] = review.find("span", class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text
            review_data["Number_review"] = review.find("div", class_="styles_consumerExtraDetails__fxS4S").find("span").text
            review_data["Language"] = review.find("div", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").text
            review_data["Title"] = review.find("h2", class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text
            review_data["Date_review"] = review.find("div", class_="styles_reviewHeader__iU9Px").time.text
            reply = review.find("div").find("p", class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX")
            review_data['Reply'] = reply.text if reply else None
            review_data['Date_reply'] = review.find("div", class_="styles_content__Hl2Mi").time.text if reply else None
            review_data["Rating"] = review.find("section", class_="styles_reviewContentwrapper__zH_9M").div["data-service-review-rating"]
            type_element = review.find_next("div", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__yqwWi")
            review_data["Status"] = type_element.find("span").text if type_element else None
            review_data["Experience Date"] = review.find("div", class_="styles_reviewContent__0Q2Tg").text
            
            data.append(review_data)
    
    df = pd.DataFrame(data)
    return df


In [2]:
younited_url = "https://www.trustpilot.com/review/www.younited-credit.com"
df_younited = extract_reviews(younited_url)
#df_younited.to_csv("younited.csv")

In [3]:
cofidis_url = "https://www.trustpilot.com/review/www.cofidis.fr"
df_cofidis = extract_reviews(cofidis_url)
#df_cofidis.to_csv("cofidis.csv")

In [4]:
floa_url = "https://www.trustpilot.com/review/www.floabank.fr"
df_floa = extract_reviews(floa_url)
#df_floa.to_csv("floa.csv")


In [5]:
orange_url = "https://www.trustpilot.com/review/www.orangebank.fr"
df_orange = extract_reviews(orange_url)
#df_orange.to_csv("orange.csv")

In [6]:
bourso_url = "https://www.trustpilot.com/review/boursorama-banque.com"
df_bourso = extract_reviews(bourso_url)
#df_bourso.to_csv("bourso.csv")

In [7]:
anytime_url = "https://www.trustpilot.com/review/anyti.me"
df_anytime = extract_reviews(anytime_url)
#df_anytime.to_csv("anytime.csv")

In [8]:
df_all = pd.concat([df_younited, df_cofidis, df_orange, df_floa, df_bourso, df_anytime], axis=0)

df_all = df_all.reset_index(drop=True)
df_all

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience Date
0,Younited Credit,STEPHANIE RENOUX,3 reviews,FR,Simple rapide et efficace !!,6 days ago,"Bonjour,Nous vous remercions d’avoir pris le t...",4 days ago,5,Verified,Simple rapide et efficace !!Date of experience...
1,Younited Credit,VERONIQUE,1 review,FR,TOP ++,2 days ago,"Bonjour,Nous vous remercions d’avoir pris le t...",2 days ago,5,Verified,TOP ++Efficacité et rapidité d'instruction Top...
2,Younited Credit,Sall,3 reviews,FR,Procedure top,"May 30, 2023","Bonjour, Nous vous remercions pour votre comme...","May 31, 2023",4,Verified,"Procedure topProcedure top, simple et rapide.T..."
3,Younited Credit,Soso,2 reviews,FR,Younited une procédure simple,"Feb 25, 2023","Bonjour,Nous vous remercions pour ce commentai...","Feb 27, 2023",5,Verified,Younited une procédure simpleYounited permet u...
4,Younited Credit,Jean-Paul,1 review,FR,Tout en simple et facile,"Jul 16, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Jul 17, 2023",4,Verified,Tout en simple et facileDate of experience: J...
...,...,...,...,...,...,...,...,...,...,...,...
2444,Anytime,RAPHAEL N,1 review,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"Sep 5, 2015",None,None,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...
2445,Anytime,FREDERIC LEGOUGE,7 reviews,FR,CARTE ANYTIME,"Jul 14, 2015",None,None,5,Invited,CARTE ANYTIMESuper ..............................
2446,Anytime,Mokhtar,2 reviews,FR,Anytime,"Jul 13, 2015",None,None,5,Invited,AnytimeSuper excellentDate of experience: July...
2447,Anytime,JEAN-LOUIS CATALDO,2 reviews,FR,Très intéressant,"Jul 13, 2015",None,None,4,Invited,Très intéressantUn système très bien étudiéDat...


In [9]:
df_all

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience Date
0,Younited Credit,STEPHANIE RENOUX,3 reviews,FR,Simple rapide et efficace !!,6 days ago,"Bonjour,Nous vous remercions d’avoir pris le t...",4 days ago,5,Verified,Simple rapide et efficace !!Date of experience...
1,Younited Credit,VERONIQUE,1 review,FR,TOP ++,2 days ago,"Bonjour,Nous vous remercions d’avoir pris le t...",2 days ago,5,Verified,TOP ++Efficacité et rapidité d'instruction Top...
2,Younited Credit,Sall,3 reviews,FR,Procedure top,"May 30, 2023","Bonjour, Nous vous remercions pour votre comme...","May 31, 2023",4,Verified,"Procedure topProcedure top, simple et rapide.T..."
3,Younited Credit,Soso,2 reviews,FR,Younited une procédure simple,"Feb 25, 2023","Bonjour,Nous vous remercions pour ce commentai...","Feb 27, 2023",5,Verified,Younited une procédure simpleYounited permet u...
4,Younited Credit,Jean-Paul,1 review,FR,Tout en simple et facile,"Jul 16, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Jul 17, 2023",4,Verified,Tout en simple et facileDate of experience: J...
...,...,...,...,...,...,...,...,...,...,...,...
2444,Anytime,RAPHAEL N,1 review,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"Sep 5, 2015",None,None,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...
2445,Anytime,FREDERIC LEGOUGE,7 reviews,FR,CARTE ANYTIME,"Jul 14, 2015",None,None,5,Invited,CARTE ANYTIMESuper ..............................
2446,Anytime,Mokhtar,2 reviews,FR,Anytime,"Jul 13, 2015",None,None,5,Invited,AnytimeSuper excellentDate of experience: July...
2447,Anytime,JEAN-LOUIS CATALDO,2 reviews,FR,Très intéressant,"Jul 13, 2015",None,None,4,Invited,Très intéressantUn système très bien étudiéDat...


In [10]:
df_all.isnull().sum()

Company              0
Customer             0
Number_review        0
Language             0
Title                0
Date_review          0
Reply              973
Date_reply         973
Rating               0
Status             111
Experience Date      0
dtype: int64

In [11]:
df_all[["Number_review", "review"]]= df_all["Number_review"].str.split(" ", n = 1, expand = True)
df_all[["Number_review", "review"]]

,Number_review,review
0,3,reviews
1,1,review
2,3,reviews
3,2,reviews
4,1,review
...,...,...
2444,1,review
2445,7,reviews
2446,2,reviews
2447,2,reviews


In [12]:
df_all[["ExperienceDate", "Date_experience"]]= df_all["Experience Date"].str.split(":", n = 1, expand = True)
df_all[["ExperienceDate", "Date_experience"]]

,ExperienceDate,Date_experience
0,Simple rapide et efficace !!Date of experience,"July 11, 2023"
1,TOP ++Efficacité et rapidité d'instruction Top...,"July 19, 2023"
2,"Procedure topProcedure top, simple et rapide.T...","May 21, 2023"
3,Younited une procédure simpleYounited permet u...,"February 19, 2023"
4,Tout en simple et facileDate of experience,"July 10, 2023"
...,...,...
2444,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"September 05, 2015"
2445,CARTE ANYTIMESuper ..............................,"July 14, 2015"
2446,AnytimeSuper excellentDate of experience,"July 13, 2015"
2447,Très intéressantUn système très bien étudiéDat...,"July 13, 2015"


In [30]:
df_all[["Experience", "Date"]]= df_all["Experience Date"].str.split("Date", n = 1, expand = True)
df_all['Experience'][2]

'Procedure topProcedure top, simple et rapide.Tt est nickel. A part les interets qui sont élevés'

In [14]:
new_df = df_all[["Company", "Customer", "Number_review", "Language", "Title", "Date_review", "Reply", "Date_reply", "Rating", "Status", "Experience", "Date_experience"]]
new_df

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
0,Younited Credit,STEPHANIE RENOUX,3,FR,Simple rapide et efficace !!,6 days ago,"Bonjour,Nous vous remercions d’avoir pris le t...",4 days ago,5,Verified,Simple rapide et efficace !!,"July 11, 2023"
1,Younited Credit,VERONIQUE,1,FR,TOP ++,2 days ago,"Bonjour,Nous vous remercions d’avoir pris le t...",2 days ago,5,Verified,TOP ++Efficacité et rapidité d'instruction Top...,"July 19, 2023"
2,Younited Credit,Sall,3,FR,Procedure top,"May 30, 2023","Bonjour, Nous vous remercions pour votre comme...","May 31, 2023",4,Verified,"Procedure topProcedure top, simple et rapide.T...","May 21, 2023"
3,Younited Credit,Soso,2,FR,Younited une procédure simple,"Feb 25, 2023","Bonjour,Nous vous remercions pour ce commentai...","Feb 27, 2023",5,Verified,Younited une procédure simpleYounited permet u...,"February 19, 2023"
4,Younited Credit,Jean-Paul,1,FR,Tout en simple et facile,"Jul 16, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Jul 17, 2023",4,Verified,Tout en simple et facile,"July 10, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...
2444,Anytime,RAPHAEL N,1,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"Sep 5, 2015",None,None,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"September 05, 2015"
2445,Anytime,FREDERIC LEGOUGE,7,FR,CARTE ANYTIME,"Jul 14, 2015",None,None,5,Invited,CARTE ANYTIMESuper ..............................,"July 14, 2015"
2446,Anytime,Mokhtar,2,FR,Anytime,"Jul 13, 2015",None,None,5,Invited,AnytimeSuper excellent,"July 13, 2015"
2447,Anytime,JEAN-LOUIS CATALDO,2,FR,Très intéressant,"Jul 13, 2015",None,None,4,Invited,Très intéressantUn système très bien étudié,"July 13, 2015"


In [15]:
new_df.groupby(by="Status").describe()

Company                              Customer         \
             count unique              top freq    count unique   
Status                                                            
Invited       1786      6      Orange Bank  730     1786   1667   
Redirected      13      1         Floabank   13       13     12   
Verified       539      2  Younited Credit  537      539    415   

                                  Number_review         ... Rating        \
                         top freq         count unique  ...    top  freq   
Status                                                  ...                
Invited               Client    9          1786     22  ...      5  1472   
Redirected  Peter Mozelewski    2            13      4  ...      5     9   
Verified             cliente   55           539     12  ...      5   463   

           Experience         \
                count unique   
Status                         
Invited          1786   1543   
Redirected         13     13   
Verified          539    452   

                                                                    \
                                                          top freq   
Status                                                               
Invited                                      Simple et rapide   29   
Redirected  Do not touch this companyDo not touch this com...    1   
Verified                                     Simple et rapide   22   

           Date_experience                                  
                     count unique                 top freq  
Status                                                      
Invited               1786    718   February 22, 2022   30  
Redirected              13     13       July 14, 2023    1  
Verified               539    436     August 09, 2020    7  

[3 rows x 44 columns]

In [16]:
new_df[["Reply", "Date_reply"]].isna().sum()


Reply         973
Date_reply    973
dtype: int64

In [17]:
new_df[["Status"]] = new_df[["Status"]].fillna(method="ffill")
new_df[["Status"]]

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Status
0,Verified
1,Verified
2,Verified
3,Verified
4,Verified
...,...
2444,Invited
2445,Invited
2446,Invited
2447,Invited


In [18]:

#new_df[new_df["Reply"] == "No Reply"]

In [19]:
new_df.isna().sum()

Company              0
Customer             0
Number_review        0
Language             0
Title                0
Date_review          0
Reply              973
Date_reply         973
Rating               0
Status               0
Experience           0
Date_experience      0
dtype: int64

In [20]:
new_df.dtypes

Company            object
Customer           object
Number_review      object
Language           object
Title              object
Date_review        object
Reply              object
Date_reply         object
Rating             object
Status             object
Experience         object
Date_experience    object
dtype: object

In [21]:
new_df[["Number_review", "Rating"]] = new_df[["Number_review", "Rating"]].astype("int64")
new_df.dtypes

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Company            object
Customer           object
Number_review       int64
Language           object
Title              object
Date_review        object
Reply              object
Date_reply         object
Rating              int64
Status             object
Experience         object
Date_experience    object
dtype: object

In [22]:
import pandas as pd
import dateparser

# Convertir les colonnes "Date_review", "Date_reply" et "Date_experience" en dates
new_df["Date_review"] = new_df["Date_review"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)

# Convertir la colonne "Date_experience" en date
new_df["Date_experience"] = new_df["Date_experience"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)

# Vérifier le résultat
new_df


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9652/319630726.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Date_review"] = new_df["Date_review"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
C:\Users\yaya-\AppData\Local\Temp/ipykernel_9652/319630726.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Date_experience"] = new_df["Date_experience"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)


,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
0,Younited Credit,STEPHANIE RENOUX,3,FR,Simple rapide et efficace !!,2023-07-22 01:19:07.047600,"Bonjour,Nous vous remercions d’avoir pris le t...",4 days ago,5,Verified,Simple rapide et efficace !!,2023-07-11
1,Younited Credit,VERONIQUE,1,FR,TOP ++,2023-07-26 01:19:07.050695,"Bonjour,Nous vous remercions d’avoir pris le t...",2 days ago,5,Verified,TOP ++Efficacité et rapidité d'instruction Top...,2023-07-19
2,Younited Credit,Sall,3,FR,Procedure top,2023-05-30 00:00:00.000000,"Bonjour, Nous vous remercions pour votre comme...","May 31, 2023",4,Verified,"Procedure topProcedure top, simple et rapide.T...",2023-05-21
3,Younited Credit,Soso,2,FR,Younited une procédure simple,2023-02-25 00:00:00.000000,"Bonjour,Nous vous remercions pour ce commentai...","Feb 27, 2023",5,Verified,Younited une procédure simpleYounited permet u...,2023-02-19
4,Younited Credit,Jean-Paul,1,FR,Tout en simple et facile,2023-07-16 00:00:00.000000,"Bonjour,Nous vous remercions d’avoir pris le t...","Jul 17, 2023",4,Verified,Tout en simple et facile,2023-07-10
...,...,...,...,...,...,...,...,...,...,...,...,...
2444,Anytime,RAPHAEL N,1,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,2015-09-05 00:00:00.000000,None,None,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,2015-09-05
2445,Anytime,FREDERIC LEGOUGE,7,FR,CARTE ANYTIME,2015-07-14 00:00:00.000000,None,None,5,Invited,CARTE ANYTIMESuper ..............................,2015-07-14
2446,Anytime,Mokhtar,2,FR,Anytime,2015-07-13 00:00:00.000000,None,None,5,Invited,AnytimeSuper excellent,2015-07-13
2447,Anytime,JEAN-LOUIS CATALDO,2,FR,Très intéressant,2015-07-13 00:00:00.000000,None,None,4,Invited,Très intéressantUn système très bien étudié,2015-07-13


In [23]:
import pandas as pd
import dateparser

#Convertir la colonne "Date_reply" en dates
new_df["Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)

#Remplacer les valeurs manquantes de "Date_reply" par les valeurs de "Date_review"
new_df["Date_reply"] = new_df.apply(lambda row: row["Date_review"] if pd.isna(row["Date_reply"]) else row["Date_reply"], axis=1)


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9652/3246385700.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
C:\Users\yaya-\AppData\Local\Temp/ipykernel_9652/3246385700.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Date_reply"] = new_df.apply(lambda row: row["Date_review"] if pd.isna(row["Date_reply"]) else row["Date_reply"], axis=1)


import pandas as pd
import dateparser

# Convert "Date_review" column to dates
new_df.loc[:, "Date_review"] = new_df["Date_review"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)

# Convert "Date_reply" column to dates
new_df.loc[:, "Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)

# Convert "Date_experience" column to dates
new_df.loc[:, "Date_experience"] = new_df["Date_experience"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)


import pandas as pd
import dateparser

Remplacer les valeurs manquantes (NaN) de la colonne "Date_reply" par celles de la colonne "Date_review"
new_df["Date_reply"].fillna(new_df["Date_review"], inplace=True)

Convertir les colonnes "Date_review", "Date_reply" et "Date_experience" en dates
new_df["Date_review"] = new_df["Date_review"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
new_df["Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
new_df["Date_experience"] = new_df["Date_experience"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
new_df.dtypes

In [24]:
new_df.isnull().sum()


Company              0
Customer             0
Number_review        0
Language             0
Title                0
Date_review         31
Reply              973
Date_reply          18
Rating               0
Status               0
Experience           0
Date_experience     29
dtype: int64

In [25]:
new_df[["Reply"]] = new_df[["Reply"]].fillna("No Reply")
#new_df[["Date_review", "Date_reply", "Date_experience"]] = new_df[["Date_review", "Date_reply", "Date_experience"]].fillna(0)
new_df.dtypes

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Company                    object
Customer                   object
Number_review               int64
Language                   object
Title                      object
Date_review        datetime64[ns]
Reply                      object
Date_reply         datetime64[ns]
Rating                      int64
Status                     object
Experience                 object
Date_experience    datetime64[ns]
dtype: object

In [26]:
new_df.isnull().sum()


Company             0
Customer            0
Number_review       0
Language            0
Title               0
Date_review        31
Reply               0
Date_reply         18
Rating              0
Status              0
Experience          0
Date_experience    29
dtype: int64

In [27]:
new_df[['Date_reply']].isnull().sum()
#satisfaction_client = new_df.dropna(how="any")
new_df.isna().sum()


Company             0
Customer            0
Number_review       0
Language            0
Title               0
Date_review        31
Reply               0
Date_reply         18
Rating              0
Status              0
Experience          0
Date_experience    29
dtype: int64

In [28]:
#new_df.to_csv("satisfaction_client.csv")